<a href="https://colab.research.google.com/github/hirokihanawa/motion-analysis/blob/master/examples/COLAB/COLAB_YOURDATA_TrainNetwork_VideoAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepLabCut Toolbox - Colab for standard (single animal) projects!
https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

### This notebook assumes you already have a project folder with labeled data!

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"

As the COLAB environments were updated to CUDA 12.X and Python 3.11, we need to install DeepLabCut and TensorFlow in a distinct way to get TensorFlow to connect to the GPU.

In [1]:
# Install TensorFlow, tensorpack and tf_slim versions compatible with DeepLabCut
!pip install "tensorflow==2.12.1" "tensorpack>=0.11" "tf_slim>=1.1.0"

In [3]:
# Downgrade PyTorch to a version using CUDA 11.8 and cudnn 8
# This will also install the required CUDA libraries, for both PyTorch and TensorFlow
!pip install torch==2.3.1 torchvision --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [1]:
# Install the latest version of DeepLabCut
!pip install "git+https://github.com/DeepLabCut/DeepLabCut.git"

  Cloning https://github.com/DeepLabCut/DeepLabCut.git to /tmp/pip-req-build-4yvdqd50
  Running command git clone --filter=blob:none --quiet https://github.com/DeepLabCut/DeepLabCut.git /tmp/pip-req-build-4yvdqd50
  Resolved https://github.com/DeepLabCut/DeepLabCut.git to commit fa2ebee4bf9b9723cf32e0a1ffe664de95b9b2e1
  Preparing metadata (setup.py) ... done


In [2]:
# As described in https://www.tensorflow.org/install/pip#step-by-step_instructions,
# create symbolic links to NVIDIA shared libraries:
!ln -svf /usr/local/lib/python3.11/dist-packages/nvidia/*/lib/*.so* /usr/local/lib/python3.11/dist-packages/tensorflow

'/usr/local/lib/python3.11/dist-packages/tensorflow/libcublasLt.so.11' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libcublasLt.so.11'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libcublasLt.so.12' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libcublasLt.so.12'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libcublas.so.11' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libcublas.so.11'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libcublas.so.12' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libcublas.so.12'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libnvblas.so.11' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libnvblas.so.11'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libnvblas.so.12' -> '/usr/local/lib/python3.11/dist-packages/nvidia/cublas/lib/libnvblas.so.12'
'/usr/local/lib/python3.11/dist-packages/tensorflow/libcheckpoint.so' -> '/usr/local/lib/python3.11/dist

**(Be sure to click "RESTART RUNTIME" if it is displayed above before moving on !)**

## Link your Google Drive (with your labeled data, or the demo data):

### First, place your project folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [1]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [ ]:
#Setup your project variables:
# PLEASE EDIT THESE:

ProjectFolderName = 'demo-demo-2025-02-18'
VideoType = 'mp4'

#don't edit these:
videofile_path = ['/content/drive/My Drive/deeplabcut_demo/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path

In [ ]:
import deeplabcut

In [ ]:
deeplabcut.__version__

In [ ]:
#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before:
path_config_file = '/content/drive/My Drive/'+ProjectFolderName+'/config.yaml'
path_config_file

## Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
# Note: if you are using the demo data (i.e. examples/Reaching-Mackenzie-2018-08-30/), first delete the folder called dlc-models!
#Then, run this cell. There are many more functions you can set here, including which netowkr to use!
#check the docstring for full options you can do!
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU...
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/wiki/DOCSTRINGS#train_network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=500)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations).
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler,
#so be sure your labels are good! (And you have trained enough ;)

## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos...
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [ ]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType)